In [145]:
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio as rio # open Tif Files
import os
#import earthpy.plot as ep
import plotly.graph_objects as go
import matplotlib.pyplot as plt
#%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot
from plotly.graph_objs import *
from rasterio.plot import show
from rasterio.mask import mask

import requests
import json

In [146]:
street_name = input("Street name: ")

Street name: Sint-Pietersvliet


In [ ]:
house_number = int(input("House number: "))

In [ ]:
post_code = int(input("Post code: "))

In [ ]:
address = requests.get("https://api.basisregisters.vlaanderen.be/v1/adresmatch",
                     params={"postcode": post_code, "straatnaam": street_name, "huisnummer":house_number})
request = json.loads(address.content)

In [ ]:
objectid = request['adresMatches'][0]['adresseerbareObjecten'][0]['objectId']

In [ ]:
building_cordinate = request['adresMatches'][0]['adresPositie']['point']['coordinates']
#building_cordinate

In [ ]:
request = requests.get(f"https://api.basisregisters.vlaanderen.be/v1/gebouweenheden/{objectid}")
building_unit = json.loads(request.content)

In [ ]:
building_id =  building_unit["gebouw"]["objectId"]
#building_id

In [ ]:
request = requests.get(f"https://api.basisregisters.vlaanderen.be/v1/gebouwen/{building_id}")
building_unit_request = json.loads(request.content)
#building_unit_request

In [ ]:
bounds_api = building_unit_request['geometriePolygoon'].get('polygon')

In [ ]:
#poly = Polygon(cordinates[0])
#poly

In [ ]:
dsm_tiff = rio.open("C:\BecodeTheano4\Assets\DSM\GeoTIFF\DHMVIIDSMRAS1m_k15.tif")

In [ ]:
dtm_tiff = rio.open("C:\BecodeTheano4\Assets\DTM\GeoTIFF\DHMVIIDTMRAS1m_k15.tif")

In [ ]:
with dsm_tiff as src:
    tif15_image_DSM, out_transform = rio.mask.mask(src, [bounds_api], crop=True)
    DSM=tif15_image_DSM[0]

In [ ]:
with dtm_tiff as src:
    tif15_image_DTM, out_transform = rio.mask.mask(src, [bounds_api], crop=True)
    DTM=tif15_image_DTM[0]

In [ ]:
CHM_15= DSM-DTM

In [ ]:
#ep.plot_bands(CHM_15, cmap='terrain')

In [ ]:
fig = go.Figure(data=go.Surface(z=CHM_15))
fig.show()